# DATA512 Project: Reliability metrics for King County Metro bus service

## Introduction

Does it feel like buses are often late or never shows up because it left earlier than expected? For users of bus transit systems in Seattle, buses that cannot be relied upon to be on-time leads to anxiety about making tight connections and as a result might intead choose to commute by car or other means that are less efficient than public mass transit.

King County Metro does measure the on-time reliability of its buses, and makes their performance measurements publicly available within its [2018 System Evaluation](https://kingcounty.gov/depts/transportation/metro/about/accountability-center/performance/route-performance.aspx). Citing traffic congestion and high ridership as contributing factors to unreliability, these metrics were used to identify routes needing additional investment to reach the system's goal. Therefore, we should examine how these metrics were calculated to better understand if these resultant service changes do improve transit rider's experience.

Sound Transit, another agency that operates bus services in the Seattle area, also publishes service quality metrics by route for its buses in its [2019 Service Implementation Plan](https://www.soundtransit.org/get-to-know-us/documents-reports/2019-service-implementation-plan-draft) as part of the planning process to determine if service changes are needed. While King County Metro's goal is for each route to have fewer than 20 percent of recorded late arrivals for all-day measures or fewer than 35 percent for the weekday PM peak period, Sound Transit in their [service standards and performance measures](https://www.soundtransit.org/get-to-know-us/documents-reports/2018-service-standards-performance-measures) quotes what seems to be a more stringent threshold of less than 15% of late bus trips overall. Is it more stringent? What conditions must be met for are buses to be deemed "on-time", and does this match transit riders' expectations?

King County Metro considers a stop to be on-time if the bus arrived at its stop from 1.5 minutes before to 5.5 minutes after its scheduled time, and each of these stops are tallied into an on-time performance metric. On the other hand, Sound Transit's performance indicator is based on punctuality of an overall bus trip, and requires a bus trip to depart no more than 3 minutes late from its start, no more than 5 minutes late from the route's midpoint and arrive no more than 7 minutes late at its terminus. At all of these checkpoints the buses must never depart early. These trips are then aggregated into an overall performance measure.

Despite both being a commuter bus service in Seattle, their on-time reliability metrics are markedly different. Why is this so, and how might the reported metrics change if redefined what it means for a bus route to be considered on-time? This project aims to augment King County Metro's system evaluation report with an alternative set of measures, such as using the evaluation criteria used by Sound Transit or another set of measures that better align with users of the transit system.

## Background

Reliability of a bus service is commonly defined as given by Abkowitz (1978) and Polus (1978) as service consistency over time and to measure service variability. When Turnquist (1982) proposed strategies for improving reliability of bus transit services, he distinguished between services that are frequent, where passengers are assumed to arrive randomly in time at a bus stop without regard to service schedule, and services with large headway between each trip, where passengers tend to learn the schedule and arrive close to the scheduled time. For measuring reliability of frequent services, the most important metric was service regularity to minimise variance, while for the infrequent services it was more important to adhere to a realistic schedule and not to depart early from checkpoints.

In addition to sampling on-time performance and service regularity, Nakanishi (1997) proposed also using door-to-door travel time  for measuring the reliability of New York City Transit services, although it also suggests that having precise, GPS-based automated data collection would also greatly improve the usefulness of on-time performance and service regularity measures.

Since the precision of non-military GPS was improved in 2000, it became common for transit agencies to have Automatic Vehicle Location systems with precise data. Trompet et al (2011) tested four service regularity metrics and found that measuring Excess Wait Time, which is the difference between actual wait time and the scheduled wait time based on scheduled headway, was the only evaluation method that best incorporates user experience. In their study, they showed that it was feasible to compare service performance between agencies using common metrics.

De Sousa et al. (2015) also reviewed evaluation metrics, and suggested that a new reliability metric dependent on service frequency is needed, and can based on the four main indicators originally proposed by Polus (1978): overall travel time, congestion index, overall travel speed and delay.

It is clear that there is no single metric that best represents service reliability for all bus services, so it would be useful to test how the metric definitions would change the reported reliability. It seems that frequent services are better measured in terms of headway and for infrequent services a schedule based measure is more appropriate. Also for infrequent services, it was important for services to not depart early for checkpoints since passengers are likely to have learned the timetable and would often arrive at the stop shortly before scheduled.

King County Metro doesn't evaluate on-time reliability in terms of headway or excess wait time, even for their frequent bus routes. Their metric for on-time reliability is a service statistic for adherence to schedule. If we adjusted the evaluation criteria in the interest of riders of the transit system, we should expect the reported metrics to worsen. But by how much?

## Methods

The premise of this project is simply to test what happens to the system evaluation measures if we adjusted their definition. This is because bad measures that don't account for transit users' experience can be deceptive if a bus route were reported as being mostly on-time, but the definition of on-time doesn't match what transit riders would consider as on-time. Since these metrics also inform allocation of investments, it's important that the right metrics are used.

### Data source

Sound Transit manages an [open transit data](https://www.soundtransit.org/help-contacts/business-information/open-transit-data-otd) platform with publicly accessible data for the Puget Sound region, including for King County Metro, and it is available via the OneBusAway API. According to Sound Transit's Open Transit Data GitHub page, the platform offers both static data such as schedules, routes and fares as well as dynamic transit data such as real-time updates and predictive vehicle location information. This [terms of use](https://www.soundtransit.org/help-contacts/business-information/open-transit-data-otd/transit-data-terms-use) allow many uses of this data including their display and analysis.

Two types of data are available in General Transit Feed Specification (GTFS) format via the [OneBusAway API](http://pugetsound.onebusaway.org/p/OneBusAwayApiService.action): static data that describes the bus routes, trips, stops and the timetable; and realtime data for vehicle locations and estimated bus arrival delays at each stop. Since GTFS realtime data is not publicly archived, I need to periodically query the API and save the results for later processing. GTFS static data is easily retrieved since they are delivered in a ZIP file, and changes less often than realtime data.

To collect realtime data, I deployed an [Azure Function](https://docs.microsoft.com/en-us/azure/azure-functions/) to query OneBusAway for King County Metro's realtime updates and save the response in protocol buffers format into [Azure Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/). This function is scheduled to run every minute so that I could get the latest bus delay update. This is akin to opening the OneBusAway app and refreshing the feed every minute and looking at all of the bus delay updates for all stops in the service area.

### Data processing

Computing the bus timetable from GTFS static is relative simple, since the data is delivered as several normalised tables in CSV format. For realtime data, I first decoded all of the trip update messages to find the value of estimated bus delay time for each bus trip and stop. Since it is common to receive multiple estimated bus delay times for the same bus trip and stop over consecutive minutes' of API requests, I need to ensure that I only keep the most recent update for each. To do this, I applied distance-based hierarchical clustering over the feed timestamp (i.e. time between requests is the distance) to identify entries that belong to the same trip and stop on the same date, as opposed to this trip and stop on a different date. The threshold was set to 3 hours, assuming that for the same trip and stop, the bus delay time would not exceed this amount.

After cleaning, I would have the last reported delay in seconds, for each stop of each bus trip and for each day's instance of the trip. I can then use this to compute the on-time reliability metric based on King County Metro's current metric as well as an alternative metric.

## Findings

For this project, I looked at making only one change: to consider buses that departed from their scheduled stops as being not reliable, instead of allowing departures of up to 1.5 minutes early. The change in metric can also be interpreted as the proportion of bus trip stops that departed from on schedule to up to 1.5 minutes early.

The on-time reliability metric, after redefining the notion of on-time, was deteriorated by up to 17%. This is a substantial amount, given that the system goals are to have fewer than 20% late stops.

### Worst 10 routes by change in lateness measure after redefinition

| route_short_name | weekday_late | saturday_late | sunday_late | alldays_late |
|------------------|--------------|---------------|-------------|--------------|
| 886              | 0.173913     | 0             | NaN         | 0.17316      |
| 888              | 0.173171     | 0.071429      | NaN         | 0.169811     |
| F Line           | 0.146499     | 0.154262      | 0.211179    | 0.155628     |
| 346              | 0.15305      | 0.158887      | 0.157125    | 0.1543       |
| 204              | 0.145154     | 0.207692      | NaN         | 0.148826     |
| 73               | 0.163368     | 0.115978      | 0.117381    | 0.145313     |
| 164              | 0.143931     | 0.158107      | 0.097054    | 0.143747     |
| 128              | 0.141596     | 0.129489      | 0.16104     | 0.142615     |
| C Line           | 0.139406     | 0.143745      | 0.143229    | 0.140433     |
| 169              | 0.140528     | 0.135224      | 0.089426    | 0.135743     |

The routes whose metrics worsened the most are 886 and 888, both of which are custom bus routes for school days, followed by RapidRide F line. From background research, early departures are less important for frequent services so for the RapidRide routes, it would be worthwhile to look at expected wait time instead.

The other routes on this table: 346, 204, 73, etc have schedules that run every half hour during peak times. Since they are not frequent services transit users are likely to learn the bus schedules and so it is important for these to not depart early.

### Best 10 routes by change in lateness measure after redefinition

| route_short_name | weekday_late | saturday_late | sunday_late | alldays_late |
|------------------|--------------|---------------|-------------|--------------|
| 982              | 0.017442     | 0             | NaN         | 0.016949     |
| 987              | 0.02439      | 0             | NaN         | 0.023179     |
| 893              | 0.025735     | NaN           | NaN         | 0.025735     |
| 119              | 0.026126     | 0.035714      | NaN         | 0.026818     |
| 986              | 0.031056     | 0             | NaN         | 0.029762     |
| 988              | 0.026667     | 0.133333      | NaN         | 0.031746     |
| 661              | 0.032877     | 0             | NaN         | 0.031746     |
| 113              | 0.035222     | 0.018072      | NaN         | 0.033288     |
| 208              | 0.037742     | 0.022315      | 0.017647    | 0.033544     |
| 980              | 0.036364     | NaN           | NaN         | 0.036364     |

At the other end of the spectrum, we see the routes that have the least change tend to be the 980s. These routes serve the private Lakeside School and University Prep. It is also good to see route 119, a ferry shuttle service, on this list since transit riders are likely to depend on adherence to schedule for making ferry connections.

## Discussion

It must be noted that the span of this project is only 2 weeks for which data was collected, so a more accurate picture of system performance could be measured if data was collected over a longer period of time to avoid seasonality and minimise random effects.

The current processing strategy relies on the value of "delay" as reported by realtime metric. If we instead measured the buses arrival times according to the pre-planned schedules, we could have a better determination of reliability as well as being able to identify cancelled bus trips or additional bus trips that were unscheduled.

Future work could investigate switching to other evaluation metrics, such as excess wait time, for frequent bus services rather than to use only the a deviation from schedule as the on-time reliability metric.

## Conclusion

In this project we only tested a small change to the definition of a bus being on-time, and found that even a small change in definition has a large effect on the reported timeliness figures.

We see that using the right metrics matter to avoid showing deceptive numbers. This means using the right metrics depending on what really matters to the users of bus transit systems. Instead of only looking at the metric values today as is, it would be useful to have an informed discussion to align with users' expectations.

## References

Abkowitz, M. (1978) _Transit Service Reliability._, USDOT Transportation Systems Center and Multisystems, Inc., Cambridge, MA.

Moreira-Matias, L., Mendes-Moreira, J., de Sousa, J. F., Gama, J. (2015) _Improving Mass Transit Operations by Using AVL-Based Systems: A Survey,_ IEEE Transactions on Intelligent Transportation Systems, vol. 16, no. 4, 1636-1653.

J Nakanishi, Y. (1997). _Bus Performance Indicators: On-Time Performance and Service Regularity._ Transportation Research Record: Journal of the Transportation Research Board. 1571. 1-13.

Polus, A. (1978) _Modeling and measurements of bus service reliability._ Transportation Research 12(4), 253-256.

Trompet, M., Liu, X., & Graham, D. J. (2011). _Development of Key Performance Indicator to Compare Regularity of Service between Urban Bus Operators._ Transportation Research Record, 2216(1), 33–41.

Turnquist, A. (1982) _Strategies for improving bus transit service reliability,_ Transportation Research Board, Washington, DC, USA, Tech. Rep.

## Code

In [1]:
# Load Azure storage account name and account key
from src import config

## Part 1: Compute the bus stops timetable from GTFS static data
First, create the timetable based on GTFS static data. This forms a scaffold of what bus trips to expect from realtime data.

We will produce a daily bus timetimetable in a table with these columns:

| Column Name      | Description                        | Example                         |
| ---              | ---                                | ---                             |
| date             | Date of the scheduled bus service  | 2018-11-21                      |
| route_id         | ID of the bus route                | 100002                          |
| route_short_name | Bus route number                   | 10                              |
| route_desc       | Route description                  | Capitol Hill - Downtown Seattle |
| trip_id          | ID of the bus trip                 | 40983438                        |
| trip_headsign    | Text displayed on bus head sign    | Downtown Seattle                |
| stop_id          | ID of the stop for this time entry | 11370                           |
| arrival_time     | Scheduled arrival time             | 05:05:40                        |
| departure_time   | Scheduled departure time           | 05:05:40                        |
| stop_sequence    | Stop sequence number along route   | 5                               |

In [2]:
import os, errno
import requests

def create_folder(path):
    """Creates a folder if it doesn't already exist."""
    created = False
    
    try:
        os.makedirs(path)
        created = True
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
            
    return created

# Prepare the raw data directory
RAW_DATA_DIR = 'data_raw'
create_folder(RAW_DATA_DIR)

# Fetch the GTFS static timetable from Open Transit Data
STATIC_ENDPOINT = 'http://developer.onebusaway.org/tmp/sound/gtfs/modified/1_gtfs.zip'

filename = '1_gtfs.zip'
path = os.path.join(RAW_DATA_DIR, filename)

# Don't re-download if we already have the data.
if os.path.exists(path):
    print("%s already exists; skipping download." % path)
else:
    print("Fetch GTFS static data to %s" % path)
    response = requests.get(STATIC_ENDPOINT)
    with open(path, 'wb') as f:
        f.write(response.content)

data_raw\1_gtfs.zip already exists; skipping download.


In [3]:
# Extract the contents of the GTFS zip file
from zipfile import ZipFile

extract_foldername = filename.replace('.zip', '')
extract_path = os.path.join(RAW_DATA_DIR, extract_foldername)

create_folder(extract_path)
with ZipFile(path) as gtfs_zip:
    gtfs_zip.extractall(extract_path)

In [4]:
# Load the bus timetable from GTFS static files
import pandas as pd
import numpy as np

agency_feed_file = os.path.join(extract_path, 'agency.txt')
calendar_feed_file = os.path.join(extract_path, 'calendar.txt')
calendar_dates_feed_file = os.path.join(extract_path, 'calendar_dates.txt')
stops_feed_file = os.path.join(extract_path, 'stops.txt')
stop_times_feed_file = os.path.join(extract_path, 'stop_times.txt')
routes_feed_file = os.path.join(extract_path, 'routes.txt')
trips_feed_file = os.path.join(extract_path, 'trips.txt')

df_agency = pd.read_csv(agency_feed_file)
df_calendar = pd.read_csv(calendar_feed_file)
df_calendar_dates = pd.read_csv(calendar_dates_feed_file)
df_stops = pd.read_csv(stops_feed_file)
df_stop_times = pd.read_csv(stop_times_feed_file)
df_routes = pd.read_csv(routes_feed_file)
df_trips = pd.read_csv(trips_feed_file)

# Convert times
df_stop_times.arrival_time = pd.to_timedelta(df_stop_times.arrival_time)
df_stop_times.departure_time = pd.to_timedelta(df_stop_times.departure_time)

# Convert dates
df_calendar.start_date = pd.to_datetime(df_calendar.start_date, format='%Y%m%d').dt.tz_localize('US/Pacific')
df_calendar.end_date = pd.to_datetime(df_calendar.end_date, format='%Y%m%d').dt.tz_localize('US/Pacific')
df_calendar_dates.date = pd.to_datetime(df_calendar_dates.date, format='%Y%m%d').dt.tz_localize('US/Pacific')

First, determine the scheduled services based on the calendar. For this project, we scope the period to measure to be the fortnight beginning from 21 Nov 2018 because the OneBusAway API does not retain realtime data for more than a few minutes. Therefore we cannot look backwards in time but can only retrieve the current status on each API call, so will only calculate the on-time reliability from when we began periodically querying the realtime data.

For each day, create a scaffold of the scheduled trips, their stops and the scheduled times for those stops. Since we are only interested in King Country Metro, we will also filter to routes by this agency only.

In [5]:
import calendar

start_date = pd.Timestamp(year=2018, month=11, day=21, unit='D', tz='US/Pacific')
measure_days = 14

daily_services = []

for d in range(measure_days):
    # Calculate the date
    current_date = start_date + pd.DateOffset(days=d)
    day_of_week = current_date.isoweekday() # Monday=1 to Sunday=7
    
    # Look up calendar for regular bus services scheduled by day of week
    scheduled = df_calendar[(df_calendar.start_date <= current_date) &
                            (df_calendar.end_date >= current_date) &
                            (df_calendar.iloc[:, day_of_week] != 0)].service_id
    
    # Apply exceptions for added or removed services on this date
    exceptions = df_calendar_dates[df_calendar_dates.date == current_date]
    added = exceptions[exceptions.exception_type == 1].service_id
    removed = exceptions[exceptions.exception_type == 2].service_id
    
    # Compute the final scheduled services for this date
    services = pd.concat([scheduled, added])
    services = services[~services.isin(removed)]
    
    # Print progress
    print(calendar.day_name[day_of_week - 1], current_date.date())
    print("Scheduled:", list(scheduled))
    print("Added:", list(added))
    print("Removed:", list(removed))
    print("Services:", list(services))
    print("Number of trips:", df_trips[df_trips.service_id.isin(services)].shape[0])
    print()
    
    # Save as dataframe
    services = pd.DataFrame(services)
    services.loc[:, "date"] = current_date
    daily_services.append(services)
    
# Combine the computed daily services into one dataframe
daily_services = pd.concat(daily_services).reset_index(drop=True)

# Filter to services operated by King County Metro only
AGENCY_ID_KCM = "1"

daily_route_stop_times = df_routes[df_routes.agency_id == AGENCY_ID_KCM] \
    .drop(['agency_id', 'route_long_name', 'route_type', 'route_url'], axis=1) \
    .merge(df_trips, left_on='route_id', right_on='route_id', suffixes=['', '_y'], validate='1:m') \
    .drop(['trip_short_name', 'route_short_name_y', 'direction_id', 'block_id', 'shape_id', 'fare_id'], axis=1) \
    .merge(df_stop_times, left_on='trip_id', right_on='trip_id', validate='1:m') \
    .drop(['stop_headsign'], axis=1)

stops_timetable = daily_services \
    .merge(daily_route_stop_times, left_on='service_id', right_on='service_id', validate='m:m') \
    .drop('service_id', axis=1)

# Preview the timetable
stops_timetable.head()

Wednesday 2018-11-21
Scheduled: [74748, 94951, 96323, 97064, 99464]
Added: [15211, 17059, 76062, 76093, 77773, 79476, 80700, 83782, 84062, 89147, 92117]
Removed: []
Services: [74748, 94951, 96323, 97064, 99464, 15211, 17059, 76062, 76093, 77773, 79476, 80700, 83782, 84062, 89147, 92117]
Number of trips: 13776

Thursday 2018-11-22
Scheduled: []
Added: [24606, 27006]
Removed: [60851, 62562, 63385, 63641, 74748, 76062, 76093, 77773, 79476, 80700, 83782, 84062, 89147, 92117, 94951, 96323, 97064, 99464]
Services: [24606, 27006]
Number of trips: 6924

Friday 2018-11-23
Scheduled: [22316, 94951, 97064, 99464]
Added: [20340]
Removed: [60851, 62562, 63385, 63641, 76062, 76093, 77773, 79476, 80700, 83782, 84062, 89147, 92117, 96323]
Services: [22316, 94951, 97064, 99464, 20340]
Number of trips: 13372

Saturday 2018-11-24
Scheduled: [17199, 20340]
Added: []
Removed: []
Services: [17199, 20340]
Number of trips: 8753

Sunday 2018-11-25
Scheduled: [23732, 24606, 27006]
Added: []
Removed: []
Services

,date,route_id,route_short_name,route_desc,trip_id,trip_headsign,stop_id,arrival_time,departure_time,stop_sequence,shape_dist_traveled
0,2018-11-21 00:00:00-08:00,100026,128,Southcenter - Alaska Junction - Admiral District,41740620,Admiral District White Center,59316,22:42:00,22:42:00,1,0.0
1,2018-11-21 00:00:00-08:00,100026,128,Southcenter - Alaska Junction - Admiral District,41740620,Admiral District White Center,40823,22:43:59,22:43:59,22,3885.1
2,2018-11-21 00:00:00-08:00,100026,128,Southcenter - Alaska Junction - Admiral District,41740620,Admiral District White Center,40825,22:44:33,22:44:33,32,4975.7
3,2018-11-21 00:00:00-08:00,100026,128,Southcenter - Alaska Junction - Admiral District,41740620,Admiral District White Center,40826,22:44:57,22:44:57,39,5746.6
4,2018-11-21 00:00:00-08:00,100026,128,Southcenter - Alaska Junction - Admiral District,41740620,Admiral District White Center,40828,22:46:00,22:46:00,59,7779.9


## Step 2: Process the realtime delay data

Download the cached GTFS realtime data that were collected every minute.

In [6]:
import azure
from azure.storage.blob import BlockBlobService
from IPython.display import display, clear_output

CONTAINER_NAME = 'bus-realtime-snapshot'
BLOB_PREFIX = '1/'

# Enumerate all of the available GTFS realtime snapshots stored in Azure Blob
blobs = []
marker = None
blob_service = BlockBlobService(account_name=config._STORAGE_ACCOUNT_NAME, account_key=config._STORAGE_ACCOUNT_KEY)

while True:
    blobs_generator = blob_service.list_blobs(container_name=CONTAINER_NAME, prefix=BLOB_PREFIX, marker=marker)
    blobs.extend(blobs_generator)
    marker = blobs_generator.next_marker
    clear_output(wait=True)
    display("%d blobs found..." % len(blobs))
    if not marker:
        break

def blob_name_to_path(blob_name, prefix):
    """Given a GTFS realtime snapshot blob's full path, derive its local output folder and filename."""
    blob_name = blob_name.lstrip(prefix)

    # Split date portion as folder name
    assert blob_name[10] == 'T'
    date_folder = blob_name[:10]

    # Compose the output path
    return date_folder, blob_name

# Limit the blobs to download only for the measurement period
start_date = pd.Timestamp(year=2018, month=11, day=21, unit='D', tz='US/Pacific')
end_date = start_date + pd.DateOffset(days=measure_days)

start_zulu = start_date.tz_convert('UTC')
end_zulu = end_date.tz_convert('UTC')

relevant_blobs = []

for blob in blobs:
    datetime_str = blob.name.lstrip(BLOB_PREFIX).rstrip('.pb')
    blob_datetime = pd.to_datetime(datetime_str, format='%Y-%m-%dT%H-%M-%SZ', utc=True)
    if start_date < blob_datetime and blob_datetime < end_date:
        relevant_blobs.append(blob)
        
print(len(relevant_blobs), "relevant blobs")

'23974 blobs found...'

20154 relevant blobs


In [7]:
# Download the blobs if we have not yet downloaded it, and only if it is within our measurement period
import threading
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm

REALTIME_FLATTENED_FILENAME = 'flattened_updates.csv.xz'
flattened_path = os.path.join(RAW_DATA_DIR, REALTIME_FLATTENED_FILENAME)
GTFS_REALTIME_OUTPUT_DIR = os.path.join(RAW_DATA_DIR, 'realtime')

def blob_download(blob_name):
    """Worker thread that downloads a blob if not yet downloaded."""
    folder_name, file_name = blob_name_to_path(blob_name, BLOB_PREFIX)
    folder_path = os.path.join(GTFS_REALTIME_OUTPUT_DIR, folder_name)
    create_folder(folder_path)
    local_path = os.path.join(folder_path, file_name)
    if not os.path.isfile(local_path):
        blob_service.get_blob_to_path(CONTAINER_NAME, blob_name, local_path)

if not os.path.isfile(flattened_path):
    _ = Parallel(n_jobs=2)(delayed(blob_download)(blob.name)
                           for blob in tqdm(relevant_blobs, desc='Download', unit='blobs'))
else:
    print("File already exists:", flattened_path)
    print("Skip download of raw blobs for individual requests.")

Load the pre-downloaded GTFS realtime data, and process it into a format suitable for analysis:
* Decode the files in Protocol Buffers binary data format
* Flatten the trip updates into a flat table that can be easily filtered and joined

In [8]:
from google.transit import gtfs_realtime_pb2
from multiprocessing import cpu_count

def process_feed(blob_name):
    folder_name, file_name = blob_name_to_path(blob_name, BLOB_PREFIX)
    local_path = os.path.join(GTFS_REALTIME_OUTPUT_DIR, folder_name, file_name)
    feed = gtfs_realtime_pb2.FeedMessage()
    
    with open(local_path, 'rb') as f:
        feed.ParseFromString(f.read())

    # Flatten the feed into a table
    entries = []
    for entity in feed.entity:
        trip_update = entity.trip_update
        update_dict = {
            'feed_timestamp': feed.header.timestamp,
            'trip_id': trip_update.trip.trip_id,
            'route_id': trip_update.trip.route_id,
            'vehicle_id': trip_update.vehicle.id,
            'trip_update_timestamp': trip_update.timestamp,
            'delay': trip_update.delay
        }

        # stop_time_update has cardinality of "Many"
        for update in trip_update.stop_time_update:
            update_dict['arrival_time'] = update.arrival.time
            update_dict['departure_time'] = update.departure.time
            update_dict['stop_id'] = update.stop_id
            entries.append(update_dict.copy())
            
    return pd.DataFrame(entries)

all_updates = None
if not os.path.isfile(flattened_path):
    print("Process and flatten blobs into one file:", flattened_path)
    updates = Parallel(n_jobs=cpu_count())(
        delayed(process_feed)(blob.name)
        for blob in tqdm(relevant_blobs, desc='Process', unit='blob'))

    all_updates = pd.concat(updates)
    del updates
    all_updates.loc[:, 'route_id'] = pd.to_numeric(all_updates.route_id)
    all_updates.loc[:, 'trip_id'] = pd.to_numeric(all_updates.trip_id)
    all_updates.loc[:, 'stop_id'] = pd.to_numeric(all_updates.stop_id)
    all_updates.to_csv(flattened_path, compression='xz', index=False)
    print('Saved to', flattened_path)
else:
    print("Load flattened data from file:", flattened_path)
    all_updates = pd.read_csv(flattened_path)

print(all_updates.shape)
display(all_updates.head())

Process and flatten blobs into one file: data_raw\flattened_updates.csv.xz



Saved to data_raw\flattened_updates.csv.xz
(6996882, 9)


,arrival_time,delay,departure_time,feed_timestamp,route_id,stop_id,trip_id,trip_update_timestamp,vehicle_id
0,0,-107,1542787188,1542787264,102552,68063,34746343,1542787188,7222
1,0,3,1542787215,1542787264,100275,2297,40572219,1542787209,8213
2,0,-66,1542787218,1542787264,100101,22970,40954860,1542787218,2800
3,0,62,1542787332,1542787264,100193,75406,39684010,1542787207,3681
4,0,17,1542787330,1542787264,102619,59876,39169323,1542787218,6000


Since we've made regular queries for realtime updates every minute, we're most likely have some duplicate updates for the same stop. How do we deduplicate updates(by taking only the most recent one) for the same stop and time? Let's examine one: Route 48 (Mt Baker - University District) at the 15th and NE Campus Pkwy stop for an evening trip.

In [10]:
df_routes[df_routes.route_id == 100228]

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_url
117,1,100228,48,NaN,3,Mt Baker - University District,http://metro.kingcounty.gov/schedules/048/n0.html


In [11]:
df_trips[df_trips.trip_id == 40569674]

,route_id,trip_id,service_id,trip_short_name,trip_headsign,route_short_name,direction_id,block_id,shape_id,fare_id
11034,100228,40569674,99464,LOCAL,University District Central District,NaN,0,5179088,11048619,101.0


In [12]:
df_stops[df_stops.stop_id == 29440]

,stop_id,stop_name,stop_lat,stop_lon,zone_id,stop_timezone
1473,29440,15th Ave NE & NE Campus Pkwy,47.655708,-122.311974,1,America/Los_Angeles


In [13]:
all_updates[(all_updates.trip_id == 40569674) & (all_updates.stop_id == 36830)] \
    .sort_values(by='feed_timestamp')

,arrival_time,delay,departure_time,feed_timestamp,route_id,stop_id,trip_id,trip_update_timestamp,vehicle_id
199,0,107,1542855280,1542855182,100228,36830,40569674,1542855106,8043
60,0,299,1543028272,1543028162,100228,36830,40569674,1543028099,8020
59,0,299,1543028272,1543028221,100228,36830,40569674,1543028099,8020
127,0,72,1543287245,1543287121,100228,36830,40569674,1543287071,8248
132,0,72,1543287245,1543287182,100228,36830,40569674,1543287071,8248
267,0,227,1543373800,1543373702,100228,36830,40569674,1543373626,8250
258,0,142,1543373715,1543373764,100228,36830,40569674,1543373716,8250
258,0,142,1543373715,1543373823,100228,36830,40569674,1543373716,8250
370,0,-10,1543546363,1543546260,100228,36830,40569674,1543546188,8227
520,0,-2,1543632771,1543632842,100228,36830,40569674,1543632771,8053


In this example, all of the rows with the same vehicle_id are for the same days' trip updates. Vehicle 8250's estimated arrival time to this stop appeared in 3 consecutive minutes' trip update queries into OneBusAway.

## Step 3: Clean the data
First, we know that we must group by route_id, trip_id and stop_id. Unfortunately, the realtime data doesn't have a trip_date to tell us which date this was for. It seems that vehicle_id could be used, but there is a chance that the same vehicle might be used for the same trip between days. So, to deduplicate multiple departure time estimates for the same stop and the same day, we will use hierarchical clustering to group timestamps that are within 3 hours of each other (assuming that the estimated departure times for the same stop doesn't change by more than that.

In [14]:
%%time
from scipy.cluster.hierarchy import linkage, fcluster

UPDATES_CHECKPOINT = 'updates_checkpoint.csv.xz'
checkpoint_path = os.path.join(RAW_DATA_DIR, UPDATES_CHECKPOINT)
three_hours = pd.to_timedelta(3, unit='H').total_seconds()

def cluster(times):
    if len(times) == 1:
        return [1]
    
    linkage_matrix = linkage(np.reshape(times.values, (len(times), 1)))
    return fcluster(linkage_matrix, three_hours, criterion='distance')

if not os.path.exists(checkpoint_path):
    print("Perform clustering within each group, and add the cluster id to the dataframe")
    clustered = all_updates \
        .groupby(['trip_id', 'stop_id']) \
        .departure_time.transform(cluster)
    all_updates.loc[:, 'cluster'] = clustered
    print("Save to checkpoint", checkpoint_path)
    all_updates.to_csv(checkpoint_path, compression='xz', index=False)
else:
    print("Already computed clustering, load from checkpoint", checkpoint_path)
    all_updates = pd.read_csv(checkpoint_path)

display(all_updates.head(10))

Perform clustering within each group, and add the cluster id to the dataframe
Save to checkpoint data_raw\updates_checkpoint.csv.xz


,arrival_time,delay,departure_time,feed_timestamp,route_id,stop_id,trip_id,trip_update_timestamp,vehicle_id,cluster
0,0,-107,1542787188,1542787264,102552,68063,34746343,1542787188,7222,6
1,0,3,1542787215,1542787264,100275,2297,40572219,1542787209,8213,1
2,0,-66,1542787218,1542787264,100101,22970,40954860,1542787218,2800,1
3,0,62,1542787332,1542787264,100193,75406,39684010,1542787207,3681,2
4,0,17,1542787330,1542787264,102619,59876,39169323,1542787218,6000,8
5,0,180,1542787288,1542787264,102615,7040,40955598,1542787189,6116,6
6,0,276,1542787355,1542787264,100045,99264,39170991,1542787205,6985,7
7,0,368,1542787808,1542787264,100045,80720,39170905,1542787210,6818,9
8,0,287,1542787371,1542787264,100009,11140,40956758,1542787218,2702,10
9,0,454,1542787282,1542787264,100026,60920,40956691,1542787209,7070,3


Wall time: 16min 2s


In [15]:
%%time

DEDUPED_FILE = 'updates_deduped.csv.xz'
deduped_path = os.path.join(RAW_DATA_DIR, DEDUPED_FILE)

if not os.path.exists(deduped_path):
    print("For each trip/stop cluster of updates, keep only the latest")
    rows_to_keep = all_updates \
        .groupby(['trip_id', 'stop_id', 'cluster']) \
        .feed_timestamp.transform(lambda t: t.index == t.idxmax())
    
    deduplicated = all_updates[rows_to_keep]
    print("Save to checkpoint", deduped_path)
    deduplicated.to_csv(deduped_path, compression='xz', index=False)

else:
    print("Already deduped, load from checkpoint", deduped_path)
    deduplicated = pd.read_csv(deduped_path)
    
print("Original", all_updates.shape[0], "rows;",
      "Deduplicated", deduplicated.shape[0], "rows")

For each trip/stop cluster of updates, keep only the latest
Save to checkpoint data_raw\updates_deduped.csv.xz
Original 6996882 rows; Deduplicated 3653760 rows
Wall time: 20min 21s


Let's revisit the example from Route 48 again, now we expect only one update for each day's trip update.

In [16]:
# Convert timestamps
deduplicated.arrival_time.replace(0, np.nan, inplace=True)
deduplicated.departure_time.replace(0, np.nan, inplace=True)
deduplicated.loc[:, 'arrival_time'] = pd.to_datetime(deduplicated.arrival_time, unit='s', utc=True)
deduplicated.loc[:, 'departure_time'] = pd.to_datetime(deduplicated.departure_time, unit='s', utc=True)
deduplicated.loc[:, 'feed_timestamp'] = pd.to_datetime(deduplicated.feed_timestamp, unit='s', utc=True)
deduplicated.loc[:, 'trip_update_timestamp'] = pd.to_datetime(deduplicated.trip_update_timestamp, unit='s', utc=True)
deduplicated.head()

C:\Users\Edmund\Anaconda3\envs\py3\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\Edmund\Anaconda3\envs\py3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,arrival_time,delay,departure_time,feed_timestamp,route_id,stop_id,trip_id,trip_update_timestamp,vehicle_id,cluster
0,NaT,-107,2018-11-21 07:59:48+00:00,2018-11-21 08:01:04+00:00,102552,68063,34746343,2018-11-21 07:59:48+00:00,7222,6
5,NaT,180,2018-11-21 08:01:28+00:00,2018-11-21 08:01:04+00:00,102615,7040,40955598,2018-11-21 07:59:49+00:00,6116,6
7,NaT,368,2018-11-21 08:10:08+00:00,2018-11-21 08:01:04+00:00,100045,80720,39170905,2018-11-21 08:00:10+00:00,6818,9
10,NaT,903,2018-11-21 08:00:46+00:00,2018-11-21 08:01:04+00:00,100252,7380,40570021,2018-11-21 07:59:51+00:00,8068,6
13,NaT,387,2018-11-21 07:59:51+00:00,2018-11-21 08:01:04+00:00,100132,600,40569536,2018-11-21 07:59:51+00:00,7129,2


In [17]:
deduplicated[(deduplicated.trip_id == 40569674) & (deduplicated.stop_id == 36830)]

,arrival_time,delay,departure_time,feed_timestamp,route_id,stop_id,trip_id,trip_update_timestamp,vehicle_id,cluster
199,NaT,107,2018-11-22 02:54:40+00:00,2018-11-22 02:53:02+00:00,100228,36830,40569674,2018-11-22 02:51:46+00:00,8043,8
59,NaT,299,2018-11-24 02:57:52+00:00,2018-11-24 02:57:01+00:00,100228,36830,40569674,2018-11-24 02:54:59+00:00,8020,7
132,NaT,72,2018-11-27 02:54:05+00:00,2018-11-27 02:53:02+00:00,100228,36830,40569674,2018-11-27 02:51:11+00:00,8248,5
258,NaT,142,2018-11-28 02:55:15+00:00,2018-11-28 02:56:04+00:00,100228,36830,40569674,2018-11-28 02:55:16+00:00,8250,6
258,NaT,142,2018-11-28 02:55:15+00:00,2018-11-28 02:57:03+00:00,100228,36830,40569674,2018-11-28 02:55:16+00:00,8250,6
370,NaT,-10,2018-11-30 02:52:43+00:00,2018-11-30 02:51:00+00:00,100228,36830,40569674,2018-11-30 02:49:48+00:00,8227,3
520,NaT,-2,2018-12-01 02:52:51+00:00,2018-12-01 02:54:02+00:00,100228,36830,40569674,2018-12-01 02:52:51+00:00,8053,4
255,NaT,-1,2018-12-04 02:52:53+00:00,2018-12-04 02:53:03+00:00,100228,36830,40569674,2018-12-04 02:52:09+00:00,8083,1
434,NaT,44,2018-12-05 02:53:38+00:00,2018-12-05 02:55:03+00:00,100228,36830,40569674,2018-12-05 02:53:37+00:00,8011,2


Save a cleaned version of the data

In [18]:
cleaned = deduplicated.loc[:, ('route_id', 'stop_id', 'trip_id', 'departure_time', 'delay')] \
    .merge(df_routes, left_on='route_id', right_on='route_id', validate='m:1') \
    .drop(['route_id', 'agency_id', 'route_long_name', 'route_type', 'route_url'], axis=1)
cleaned.loc[:, 'weekday_name'] = cleaned.departure_time.dt.weekday_name
cleaned.head()

,stop_id,trip_id,departure_time,delay,route_short_name,route_desc,weekday_name
0,68063,34746343,2018-11-21 07:59:48+00:00,-107,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday
1,68360,34746343,2018-11-21 08:01:43+00:00,-95,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday
2,68370,34746343,2018-11-21 08:02:52+00:00,-68,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday
3,68372,34746343,2018-11-21 08:03:38+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday
4,66710,34746343,2018-11-21 08:04:34+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday


In [19]:
CLEANED_DATA_DIR = 'data_cleaned'
create_folder(CLEANED_DATA_DIR)

CLEANED_CSV_FILE = 'cleaned.csv.xz'
cleaned_csv_path = os.path.join(CLEANED_DATA_DIR, CLEANED_CSV_FILE)

if not os.path.exists(cleaned_csv_path):
    print("Save cleaned data to", cleaned_csv_path)
    cleaned.to_csv(cleaned_csv_path, compression='xz')
else:
    print("Cleaned csv", cleaned_csv_path, "already exists.")

Save cleaned data to data_cleaned\cleaned.csv.xz


## Step 4: Calculate metrics
Calculate if each stop was "on-time" by KCM standard (between 90 seconds early and 330 seconds late).

In [20]:
cleaned.loc[:, 'on_time_stop'] = (cleaned.delay >= -90) & (cleaned.delay <= 330)
cleaned.head(10)

,stop_id,trip_id,departure_time,delay,route_short_name,route_desc,weekday_name,on_time_stop
0,68063,34746343,2018-11-21 07:59:48+00:00,-107,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,False
1,68360,34746343,2018-11-21 08:01:43+00:00,-95,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,False
2,68370,34746343,2018-11-21 08:02:52+00:00,-68,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
3,68372,34746343,2018-11-21 08:03:38+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
4,66710,34746343,2018-11-21 08:04:34+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
5,66710,34746343,2018-11-21 08:04:34+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
6,66740,34746343,2018-11-21 08:05:48+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
7,67360,34746343,2018-11-21 08:08:01+00:00,-42,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
8,67360,34746343,2018-11-21 08:08:01+00:00,-42,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True
9,67380,34746343,2018-11-21 08:09:40+00:00,-42,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True


Calculate the on-time performance for weekdays, Saturday and Sunday

In [21]:
kcm_late = pd.DataFrame({
    'weekday_late': 1 - cleaned[~cleaned.weekday_name.isin(['Saturday', 'Sunday'])] \
        .groupby('route_short_name').on_time_stop.mean(),
    'saturday_late': 1 - cleaned[cleaned.weekday_name == 'Saturday'] \
        .groupby('route_short_name').on_time_stop.mean(),
    'sunday_late': 1 - cleaned[cleaned.weekday_name == 'Sunday'] \
        .groupby('route_short_name').on_time_stop.mean(),
    'alldays_late': 1 - cleaned.groupby('route_short_name').on_time_stop.mean()
})
display(kcm_late.head())

KCM_LATE_CSV = 'kcm_late.csv'
kcm_late_path = os.path.join(CLEANED_DATA_DIR, KCM_LATE_CSV)
kcm_late.to_csv(kcm_late_path, index_label='route_short_name')

,weekday_late,saturday_late,sunday_late,alldays_late
1,0.246402,0.307158,0.232231,0.253248
10,0.281471,0.265315,0.220632,0.271214
101,0.319900,0.296820,0.307197,0.316077
102,0.461422,0.482234,NaN,0.463306
105,0.252622,0.195515,0.116460,0.228765


In [22]:
cleaned.loc[:, 'on_time_stop_no_early'] = (cleaned.delay >= 0) & (cleaned.delay <= 330)
cleaned.head(10)

,stop_id,trip_id,departure_time,delay,route_short_name,route_desc,weekday_name,on_time_stop,on_time_stop_no_early
0,68063,34746343,2018-11-21 07:59:48+00:00,-107,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,False,False
1,68360,34746343,2018-11-21 08:01:43+00:00,-95,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,False,False
2,68370,34746343,2018-11-21 08:02:52+00:00,-68,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
3,68372,34746343,2018-11-21 08:03:38+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
4,66710,34746343,2018-11-21 08:04:34+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
5,66710,34746343,2018-11-21 08:04:34+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
6,66740,34746343,2018-11-21 08:05:48+00:00,-64,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
7,67360,34746343,2018-11-21 08:08:01+00:00,-42,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
8,67360,34746343,2018-11-21 08:08:01+00:00,-42,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False
9,67380,34746343,2018-11-21 08:09:40+00:00,-42,226,Eastgate P&R-Crossroads-Overlake-Bellevue TC,Wednesday,True,False


Calculate the on-time performance for weekdays, Saturday and Sunday

In [23]:
kcm_late_stricter = pd.DataFrame({
    'weekday_late': 1 - cleaned[~cleaned.weekday_name.isin(['Saturday', 'Sunday'])] \
        .groupby('route_short_name').on_time_stop_no_early.mean(),
    'saturday_late': 1 - cleaned[cleaned.weekday_name == 'Saturday'] \
        .groupby('route_short_name').on_time_stop_no_early.mean(),
    'sunday_late': 1 - cleaned[cleaned.weekday_name == 'Sunday'] \
        .groupby('route_short_name').on_time_stop_no_early.mean(),
    'alldays_late': 1 - cleaned.groupby('route_short_name').on_time_stop_no_early.mean(),
})
display(kcm_late_stricter.head())

KCM_LATE_SRTICTER_CSV = 'kcm_late_stricter.csv'
kcm_late_stricter_path = os.path.join(CLEANED_DATA_DIR, KCM_LATE_SRTICTER_CSV)
kcm_late_stricter.to_csv(kcm_late_stricter_path, index_label='route_short_name')

,weekday_late,saturday_late,sunday_late,alldays_late
1,0.360100,0.394141,0.315089,0.359861
10,0.371238,0.396651,0.381046,0.376211
101,0.405262,0.391343,0.381584,0.401572
102,0.535883,0.580372,NaN,0.539911
105,0.313185,0.315124,0.212733,0.302241


Summarise the findings

In [28]:
change = kcm_late_stricter - kcm_late

print('Routes that worsened the most:')
display(change.sort_values(by='alldays_late', ascending=False).head(10))

print('Routes that worsened the least:')
display(change.sort_values(by='alldays_late', ascending=True).head(10))

Routes that worsened the most:


,weekday_late,saturday_late,sunday_late,alldays_late
route_short_name,,,,
886,0.173913,0.000000,NaN,0.173160
888,0.173171,0.071429,NaN,0.169811
F Line,0.146499,0.154262,0.211179,0.155628
346,0.153050,0.158887,0.157125,0.154300
204,0.145154,0.207692,NaN,0.148826
73,0.163368,0.115978,0.117381,0.145313
164,0.143931,0.158107,0.097054,0.143747
128,0.141596,0.129489,0.161040,0.142615
C Line,0.139406,0.143745,0.143229,0.140433


Routes that worsened the least:


,weekday_late,saturday_late,sunday_late,alldays_late
route_short_name,,,,
982,0.017442,0.000000,NaN,0.016949
987,0.024390,0.000000,NaN,0.023179
893,0.025735,NaN,NaN,0.025735
119,0.026126,0.035714,NaN,0.026818
986,0.031056,0.000000,NaN,0.029762
988,0.026667,0.133333,NaN,0.031746
661,0.032877,0.000000,NaN,0.031746
113,0.035222,0.018072,NaN,0.033288
208,0.037742,0.022315,0.017647,0.033544


For each trip, use the timetable to find out which which stops are the start, mid-point and end.

In [24]:
# Calculate the mid-point distance on each trip
trip_max_dist = daily_route_stop_times.groupby('trip_id').shape_dist_traveled.max()
trip_mid_dist = trip_max_dist / 2

# For each trip, calculate each stop's travel distance from midpoint
dist_from_mid = np.abs(daily_route_stop_times.shape_dist_traveled
                       - trip_mid_dist[daily_route_stop_times.trip_id].reset_index(drop=True))
daily_route_stop_times.loc[:, 'dist_from_mid'] = dist_from_mid
daily_route_stop_times.head()

,route_id,route_short_name,route_desc,trip_id,service_id,trip_headsign,stop_id,arrival_time,departure_time,stop_sequence,shape_dist_traveled,dist_from_mid
0,100001,1,Kinnear - Downtown Seattle,40983896,99464,Kinnear Seattle Center W,3600,05:26:00,05:26:00,1,0.0,13352.0
1,100001,1,Kinnear - Downtown Seattle,40983896,99464,Kinnear Seattle Center W,1500,05:27:17,05:27:17,4,1128.3,12223.7
2,100001,1,Kinnear - Downtown Seattle,40983896,99464,Kinnear Seattle Center W,1510,05:28:10,05:28:10,9,1902.4,11449.6
3,100001,1,Kinnear - Downtown Seattle,40983896,99464,Kinnear Seattle Center W,1530,05:29:00,05:29:00,13,2622.5,10729.5
4,100001,1,Kinnear - Downtown Seattle,40983896,99464,Kinnear Seattle Center W,1610,05:31:17,05:31:17,22,3800.7,9551.3


In [25]:
# Calculate for each trip, the stop that is the mid-point stop
trip_midpoint = daily_route_stop_times.groupby('trip_id') \
    .dist_from_mid.transform(lambda d: d.index == d.idxmin())
midpoints = daily_route_stop_times[trip_midpoint]
midpoints = midpoints.loc[:, ('trip_id', 'stop_id')]
midpoints.head()

,trip_id,stop_id
12,40983896,2332
41,40983897,2130
60,40983900,2130
78,40983901,2360
107,40983902,2332


In [26]:
# Find the starts
trip_start = daily_route_stop_times.groupby('trip_id') \
    .shape_dist_traveled.transform(lambda d: d.index == d.idxmin())
starts = daily_route_stop_times[trip_start]
starts = starts.loc[:, ('trip_id', 'stop_id')]
starts.head()

,trip_id,stop_id
0,40983896,3600
30,40983897,2010
49,40983900,2010
68,40983901,1530
95,40983902,3600


In [27]:
# And the terminating stops
trip_end = daily_route_stop_times.groupby('trip_id') \
    .shape_dist_traveled.transform(lambda d: d.index == d.idxmax())
ends = daily_route_stop_times[trip_end]
ends = ends.loc[:, ('trip_id', 'stop_id')]
ends.head()

,trip_id,stop_id
29,40983896,2010
48,40983897,2220
67,40983900,2220
94,40983901,2010
124,40983902,2010
